In [1]:
# load the necessary python libraries
import os
import xml.etree.cElementTree as cET
from collections import defaultdict
import pprint
import re
import codecs
import string

In [5]:
# data file
OSM_FILE = "sampleK100.osm"

def count_tags(inputfile):
    tags = {}
    for _, elem in cET.iterparse(inputfile):
        tag = elem.tag
        if tag not in tags.keys():
            tags[tag] = 1
        else:
            tags[tag] += 1
    return tags




pprint.pprint(count_tags(OSM_FILE))


{'member': 30,
 'nd': 35383,
 'node': 28427,
 'osm': 1,
 'relation': 9,
 'tag': 7759,
 'way': 6526}


In [6]:
def get_user(element):
    return

def process_map(inputfile):
    users = set()
    for _, element in cET.iterparse(inputfile):
        try:
            users.add(element.attrib['uid'])
        except KeyError:
            continue
    return users

print 'Number of unique contributors:', len(process_map(OSM_FILE))

Number of unique contributors: 375


In [26]:
# some regular expression 
lower = re.compile(r'^([a-z]|_)*$') 
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# initial version of expected street names
expected_streets = ["Street", "Road", "Cross", "Main"]

In [ ]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected_streets:
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_street(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in cET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types

st_types = audit_street(OSM_FILE)
pprint.pprint(dict(st_types))

In [ ]:
# Audit PIN CODES. bangalore pin codes are of the format 56xxxx. 
#(see http://www.mapsofindia.com/pincode/india/karnataka/bangalore/)

def audit_zipcodes(osmfile):
    osm_file = open(osmfile, "r")
    zip_codes = {}
    for event, elem in cET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "addr:postcode" and not tag.attrib['v'].startswith('56'):
                    if tag.attrib['v'] not in zip_codes:
                        zip_codes[tag.attrib['v']] = 1
                    else:
                        zip_codes[tag.attrib['v']] += 1
    return zip_codes

zipcodes = audit_zipcodes(OSM_FILE)
for zipcode in zipcodes:
    print zipcode, zipcodes[zipcode]
